# Retreival System

> Aim: create a database containing previous puzzles (with there solutions) and create a retreival system that given a new puzzle finds similar puzzles and returns the (best) solutions for the similar puzzles.

In [3]:
import os
import sys

# Append the models path in order to import the models
PROJECT_ROOT = os.path.join(os.getcwd(), '../src/')
print(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

from models.gemini_model import GeminiLanguageModel

/home/twanh/workspace/thesis/thesis-advent-of-agents/experiments/../src/


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
# Create a model that can be used
model = GeminiLanguageModel(
    api_key=os.getenv("GEMINI_API_KEY"),
    model_name='gemini-2.0-flash'
)

In [5]:
# Load all puzzles
# puzzles are stored in a directory structure [year]/[day].txt
PUZZLES_PATH = '/home/twanh/workspace/thesis/puzzles/auto'


In [24]:
# Setup the postgres database
from pgvector.psycopg import register_vector
import psycopg

conn = psycopg.connect(
    "host=localhost dbname=advent_of_agents user=postgres password=postgres",
    autocommit=True,
)

conn.execute('CREATE EXTENSION IF NOT EXISTS vector')

register_vector(conn)

conn.execute('''
-- Puzzles table
CREATE TABLE puzzles (
  id SERIAL PRIMARY KEY,
  year INT NOT NULL,
  day INT NOT NULL,
  full_description TEXT,
  problem_statement TEXT,
  keywords TEXT,
  underlying_concepts TEXT,
  embedding VECTOR(1536),  -- OpenAI embedding dimension (small model)
  UNIQUE(year, day)  -- Ensure year/day combinations are unique
);

-- Create an index on the vector column for fast nearest-neighbor search
CREATE INDEX idx_embedding ON puzzles
  USING ivfflat (embedding vector_l2_ops) WITH (lists = 100);

-- Solutions table with a foreign key to puzzles
CREATE TABLE solutions (
  id SERIAL PRIMARY KEY,
  puzzle_id INT NOT NULL,
  code TEXT NOT NULL,
  author VARCHAR(255),
  source VARCHAR(255),
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (puzzle_id) REFERENCES puzzles(id) ON DELETE CASCADE
);

-- Index on puzzle_id for faster joins
CREATE INDEX idx_solutions_puzzle_id ON solutions(puzzle_id);
''')
conn.close()

DuplicateTable: relation "puzzles" already exists

In [7]:
# Get all the paths for the puzzles
def get_puzzle_paths(puzzles_path):
    puzzle_paths = []
    for year in os.listdir(puzzles_path):
        year_path = os.path.join(puzzles_path, year)
        if os.path.isdir(year_path):
            for day in os.listdir(year_path):
                day_path = os.path.join(year_path, day)
                if os.path.isfile(day_path):
                    if day_path.endswith('.txt'):
                        puzzle_paths.append(day_path)
    return puzzle_paths
puzzle_paths = get_puzzle_paths(PUZZLES_PATH)
print(puzzle_paths)
print(f"Found {len(puzzle_paths)} puzzle files.")

['/home/twanh/workspace/thesis/puzzles/auto/2015/day_2_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_7_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_3_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_11_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_19_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_15_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_14_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_9_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_5_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_16_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_20_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_8_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_1_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_23_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_6_

In [29]:
from dataclasses import dataclass

@dataclass
class PuzzleData:
    full_description: str
    year: int
    day: int
    problem_statement: str
    keywords: list[str]
    underlying_concepts: list[str]
    
    def __getitem__(self, item):
        return getattr(self, item)


In [56]:
import re
from pprint import pformat
from agents.pre_processing_agent import PreProcessingAgent
from core.state import MainState
from utils.util_types import Puzzle

def extract_year_day(puzzle_path: str) -> tuple[int,int]:


    pattern = r".*/(\d{4})/day_(\d+)_part_1\.txt"
    match = re.match(pattern, puzzle_path)
    if match:
        year, day = match.groups()
        return (year, day)
        

    print(f"Error: could not extract year,day from: {puzzle_path}")
    return 0, 0


def pre_process_puzzle(puzzle_path: str, year=None, day=None) -> PuzzleData:

    
    puzzle_desc = None
    if year is None and day is None:
        with open(puzzle_path, 'r') as puzzle_file:
            puzzle_desc = puzzle_file.read()

        year, day = extract_year_day(puzzle_path)
    else:
        # XXX: Should not work like this
        puzzle_desc = puzzle_path

    puzzle = Puzzle(
        description=puzzle_desc,
        day = day,
        year=year,
        solution=None
    )

    state = MainState(puzzle=puzzle)

    pre_agent = PreProcessingAgent(agent_name='pre-processing', model=model)

    updated_state = pre_agent.process(state)

    pformat(updated_state)

    return PuzzleData(
        full_description=updated_state.puzzle.description,
        year = updated_state.puzzle.year,
        day = updated_state.puzzle.day,
        problem_statement=updated_state.problem_statement,
        underlying_concepts=updated_state.underlying_concepts,
        keywords=updated_state.keywords,
    )


pre_processed_puzzles = [pre_process_puzzle(puzzle_path) for puzzle_path in puzzle_paths[:5]]

2025-04-15 11:45:48.558 | DEBUG    | agents.pre_processing_agent:process:21 - Preprocessing agent prompt: You are a pre-processing agent. Your task is to process the following Advent of Code puzzle description so that the output can be stored in a RAG (Retrieval-Augmented Generation) database and later used by a planning agent. Follow these steps precisely:

------------------------------------------------------------
Step 1: Detailed Extraction
------------------------------------------------------------
Extract every technical detail from the puzzle and separate it from the narrative. Specifically, please extract:
  - Core Problem Statement:
      - A concise description of the computational task (exclude story or decorative language).
  - Input Specifications:
     - Detailed descriptions of the expected input format, data types, ranges, and constraints.
  - Output Specifications:
      - Clear requirements on what the solution must produce (format, limits, etc.).
  - Test cases inp

In [21]:
import pprint
pprint.pprint(pre_processed_puzzles)


[PuzzleData(full_description='## \\--- Day 2: I Was Told There Would Be No '
                             'Math ---\n'
                             '\n'
                             'The elves are running low on wrapping paper, and '
                             'so they need to submit an order for more. They '
                             'have a list of the dimensions (length `l`, width '
                             '`w`, and height `h`) of each present, and only '
                             'want to order exactly as much as they need.\n'
                             '\n'
                             'Fortunately, every present is a box (a perfect '
                             '[right rectangular '
                             'prism](https://en.wikipedia.org/wiki/Cuboid#Rectangular_cuboid)), '
                             'which makes calculating the required wrapping '
                             'paper for each gift a little easier: find the '
                             'su

In [50]:
import numpy as np
import dataclasses
from openai import OpenAI

WEIGHTS = {
    'full_description': 0.1,
    'problem_statement': 0.3,
    'underlying_concepts': 0.4,
    'keywords': 0.2,
}


embedding_model = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def create_embedding(text: str):

    response = embedding_model.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )

    return response.data[0].embedding


def compute_weighted_embedding(puzzle_data: PuzzleData, weights: dict[str, float]) -> list[float]:

    puzzle_data = dataclasses.asdict(puzzle_data)
    # THis assumes that all the fields have a weight if they need to be included in the compsite embedding
    embeddings = {}
    for field in weights:
        embeddings[field] = np.array(create_embedding(puzzle_data[field]))


    composite_embedding = np.zeros_like(list(embeddings.values())[0])
    for field, weight in weights.items():
        weighted_embedding = weight * embeddings[field]
        composite_embedding += weighted_embedding

    # Normalize the embedding
    norm = np.linalg.norm(composite_embedding)
    if norm > 0:
        composite_embedding  = composite_embedding / norm

    return composite_embedding.tolist()




In [52]:
def add_puzzle_to_db(puzzle_data: PuzzleData, conn) -> int:

    embedding = compute_weighted_embedding(puzzle_data, WEIGHTS)

    with conn.cursor() as cur:

        query = """
        INSERT INTO puzzles (
            year, day, full_description, problem_statement, 
            keywords, underlying_concepts, embedding
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        RETURNING id;
        """
        
        cur.execute(query, (
            puzzle_data.year,
            puzzle_data.day,
            puzzle_data.full_description,
            puzzle_data.problem_statement,
            puzzle_data.keywords,
            puzzle_data.underlying_concepts,
            embedding
        ))
        
        puzzle_id = cur.fetchone()[0]
        conn.commit()
    
        return puzzle_id

In [53]:

conn = psycopg.connect(
    "host=localhost dbname=advent_of_agents user=postgres password=postgres",
    autocommit=True,
)

for puzzle in pre_processed_puzzles:

    pid = add_puzzle_to_db(puzzle, conn)
    print(f"Added puzzle to db with puzzle_id: {pid}")

conn.close()

Added puzzle to db with puzzle_id: 1
Added puzzle to db with puzzle_id: 2
Added puzzle to db with puzzle_id: 3
Added puzzle to db with puzzle_id: 4
Added puzzle to db with puzzle_id: 5


In [57]:
# Test Puzzle For Retreival

PUZZLE_DESC = """
You and The Historians look a lot more pixelated than you remember. You're inside a computer at the North Pole!

Just as you're about to check out your surroundings, a program runs up to you. "This region of memory isn't safe! The User misunderstood what a pushdown automaton is and their algorithm is pushing whole bytes down on top of us! Run!"

The algorithm is fast - it's going to cause a byte to fall into your memory space once every nanosecond! Fortunately, you're faster, and by quickly scanning the algorithm, you create a list of which bytes will fall (your puzzle input) in the order they'll land in your memory space.

Your memory space is a two-dimensional grid with coordinates that range from 0 to 70 both horizontally and vertically. However, for the sake of example, suppose you're on a smaller grid with coordinates that range from 0 to 6 and the following list of incoming byte positions:

5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
Each byte position is given as an X,Y coordinate, where X is the distance from the left edge of your memory space and Y is the distance from the top edge of your memory space.

You and The Historians are currently in the top left corner of the memory space (at 0,0) and need to reach the exit in the bottom right corner (at 70,70 in your memory space, but at 6,6 in this example). You'll need to simulate the falling bytes to plan out where it will be safe to run; for now, simulate just the first few bytes falling into your memory space.

As bytes fall into your memory space, they make that coordinate corrupted. Corrupted memory coordinates cannot be entered by you or The Historians, so you'll need to plan your route carefully. You also cannot leave the boundaries of the memory space; your only hope is to reach the exit.

In the above example, if you were to draw the memory space after the first 12 bytes have fallen (using . for safe and # for corrupted), it would look like this:

...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....
You can take steps up, down, left, or right. After just 12 bytes have corrupted locations in your memory space, the shortest path from the top left corner to the exit would take 22 steps. Here (marked with O) is one such path:

OO.#OOO
.O#OO#O
.OOO#OO
...#OO#
..#OO#.
.#.O#..
#.#OOOO
Simulate the first kilobyte (1024 bytes) falling onto your memory space. Afterward, what is the minimum number of steps needed to reach the exit?

"""

test_puzzle = pre_process_puzzle(PUZZLE_DESC, 2024, 18)
print(test_puzzle)

2025-04-15 11:46:07.171 | DEBUG    | agents.pre_processing_agent:process:21 - Preprocessing agent prompt: You are a pre-processing agent. Your task is to process the following Advent of Code puzzle description so that the output can be stored in a RAG (Retrieval-Augmented Generation) database and later used by a planning agent. Follow these steps precisely:

------------------------------------------------------------
Step 1: Detailed Extraction
------------------------------------------------------------
Extract every technical detail from the puzzle and separate it from the narrative. Specifically, please extract:
  - Core Problem Statement:
      - A concise description of the computational task (exclude story or decorative language).
  - Input Specifications:
     - Detailed descriptions of the expected input format, data types, ranges, and constraints.
  - Output Specifications:
      - Clear requirements on what the solution must produce (format, limits, etc.).
  - Test cases inp

2025-04-15 11:46:10.770 | DEBUG    | agents.pre_processing_agent:process:23 - Model response: ```json
{
  "problem_statement": "Find the shortest path from (0, 0) to (70, 70) on a 71x71 grid, avoiding corrupted cells. A cell becomes corrupted when a byte falls on it, and the first 1024 bytes' fall locations are given as input.",
  "input_format": "A list of 1024 coordinate pairs (x, y), where x and y are integers representing the x and y coordinates of the memory space. Each coordinate is in the range 0 to 70 inclusive. The input is given as a comma separated list with x and y.",
  "output_format": "The minimum number of steps needed to reach the exit (70, 70) from the starting point (0, 0) after 1024 bytes have fallen. Movement is allowed up, down, left, or right. Output is a single integer representing the number of steps.",
  "test_cases": [
    {
      "input": "5,4\n4,2\n4,5\n3,0\n2,1\n6,3\n2,4\n1,5\n0,6\n3,3\n2,6\n5,1",
      "output": "22"
    }
  ],
  "constraints": [
    "The 

PuzzleData(full_description='\nYou and The Historians look a lot more pixelated than you remember. You\'re inside a computer at the North Pole!\n\nJust as you\'re about to check out your surroundings, a program runs up to you. "This region of memory isn\'t safe! The User misunderstood what a pushdown automaton is and their algorithm is pushing whole bytes down on top of us! Run!"\n\nThe algorithm is fast - it\'s going to cause a byte to fall into your memory space once every nanosecond! Fortunately, you\'re faster, and by quickly scanning the algorithm, you create a list of which bytes will fall (your puzzle input) in the order they\'ll land in your memory space.\n\nYour memory space is a two-dimensional grid with coordinates that range from 0 to 70 both horizontally and vertically. However, for the sake of example, suppose you\'re on a smaller grid with coordinates that range from 0 to 6 and the following list of incoming byte positions:\n\n5,4\n4,2\n4,5\n3,0\n2,1\n6,3\n2,4\n1,5\n0,6\

In [83]:
def get_n_similar_puzzles(conn, puzzle_data: PuzzleData, n: int = 3) -> list[PuzzleData]:


    # Create embedding for the new puzzle
    embedding = compute_weighted_embedding(puzzle_data, WEIGHTS)
    print(embedding)

    with conn.cursor() as cur:
        query = """
        SELECT id, year, day, full_description, problem_statement, 
                keywords, underlying_concepts,
                embedding <-> %s::vector AS distance
        FROM puzzles
        ORDER BY distance
        LIMIT %s;
        """
        
        cur.execute(query, (embedding, n))
        results = cur.fetchall()

        print(len(results[0]))

        return [
            PuzzleData(
                year=result[1],
                day=result[2],
                full_description=result[3],
                problem_statement=result[4],
                keywords=result[5],
                underlying_concepts=result[6]
            ) for result in results
        ]

In [84]:

conn = psycopg.connect(
    "host=localhost dbname=advent_of_agents user=postgres password=postgres",
    autocommit=True,
)

results = get_n_similar_puzzles(conn, test_puzzle, n=3)
pprint.pprint(test_puzzle)
pprint.pprint(results[0])

[-0.019912083437813012, -0.002403337782691781, 0.06381189051070076, -0.010750612431284763, -0.04941439660133563, -0.03348449278862562, 0.0037672762532733094, 0.0061612565815155835, -0.03420849534719717, 0.08358307144700751, 0.06612136718988036, -0.04400967119496433, 0.02244957838325656, -0.007509867140526196, 0.034493013933847586, -0.029457933768543366, 0.024965250296498286, -0.051849940263507116, -0.03510724341795754, 0.00836717057140477, 0.038657781800675285, -0.013831651494624515, -0.0030539454130069877, 0.035720320218421184, -0.001292600449924556, -0.0036979919489902977, 0.050278395483648895, 0.03757399996748741, 0.05733278162537808, -0.04437816863691382, 0.011644303016253599, -0.032967131063387785, -0.02776612345812162, -0.01355594314867328, 0.0441491953475983, 0.0297861709711304, -0.033699637912062465, 0.03249303590723709, -0.003244281950046053, 0.024619992204692683, -0.010737978044220543, -0.002102609985238758, 0.0033929124272272693, -0.03033233288705668, 0.029746717134892905, 0